# Convolutional Neural Networks

## Import libraries

In [1]:
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import wget
import numpy as np
import pandas as pd
import shutil
import zipfile
import warnings
from distutils.dir_util import copy_tree
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

Using TensorFlow backend.


## Download and extract dataset

In [2]:
if os.path.exists('data'):
    print("Path already exists")
else:
    print("Creating the data directory")
    os.mkdir('data')
    print("Directory created")

Path already exists


In [3]:
data_url = 'https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P16-Convolutional-Neural-Networks.zip'
file_name = data_url.split("/")[-1]
if not os.path.exists('data/training_set'):
    print('Need to download dataset')
    wget.download(data_url, out = 'data')
    print('Data downloaded')
    with zipfile.ZipFile('data/' + file_name, 'r') as file:
        print("Extracting dataset")
        file.extractall()
        copy_tree('Convolutional_Neural_Networks/dataset/', 'data/')
        print("Dataset extracted and created")
    try:
        shutil.rmtree('Convolutional_Neural_Networks')
        os.remove('data/' + file_name)
        print("Extra files removed")
    except:
        print("Could not delete extra files")
else:
    print('Dataset available')

Dataset available


## Building the model

In [4]:
classifier = Sequential()
classifier.add(Convolution2D(filters = 32, kernel_size = (3, 3), input_shape = (3, 64, 64), activation = 'relu', data_format = 'channels_first'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 62, 62)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 31, 31)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30752)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3936384   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 3,937,409
Trainable params: 3,937,409
Non-trainable params: 0
_________________________________________________________________


## Data Augmentation

In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(rescale = 1.0/255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255)

training_set = train_datagen.flow_from_directory('data/training_set', 
                                                 target_size = (64, 64), 
                                                 batch_size = 32, 
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('data/test_set', 
                                            target_size = (64, 64), 
                                            batch_size = 32, 
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 8000/32,
                         epochs = 25, 
                         validation_data = test_set,
                         validation_steps = 2000/32,
                         verbose = 1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/25
250/250 [==============================] - 261s 1s/step - loss: 0.6503 - acc: 0.6262 - val_loss: 0.5862 - val_acc: 0.6900
Epoch 2/25
250/250 [==============================] - 80s 319ms/step - loss: 0.5837 - acc: 0.6935 - val_loss: 0.5805 - val_acc: 0.7090
Epoch 3/25
250/250 [==============================] - 82s 328ms/step - loss: 0.5540 - acc: 0.7139 - val_loss: 0.5532 - val_acc: 0.7170
Epoch 4/25
250/250 [==============================] - 81s 323ms/step - loss: 0.5367 - acc: 0.7285 - val_loss: 0.5493 - val_acc: 0.7260
Epoch 5/25
250/250 [==============================] - 77s 309ms/step - loss: 0.5108 - acc: 0.7478 - val_loss: 0.5473 - val_acc: 0.7375
Epoch 6/25
250/250 [==============================] - 74s 294ms/step - loss: 0.4984 - acc: 0.7521 - val_loss: 0.6242 - val_acc: 0.6710
Epoch 7/25
250/250 [==============================] - 78s 310ms/step - loss: 0.4830 - acc: 0.7652 - val_loss: 0.5084 - val_acc: 0.7745
Epoch 8/2

## Improving the model to get test accuracy around 80%

In [8]:
classifier = Sequential()
classifier.add(Convolution2D(filters = 32, kernel_size = (3, 3), input_shape = (3, 64, 64), activation = 'relu', data_format = 'channels_first'))
classifier.add(Convolution2D(filters = 32, kernel_size = (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 62, 62)        896       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 60, 60)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 30, 30)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               3686528   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 3,696,801
Trainable params: 3,696,801
Non-trainable params: 0
_________________________________________________________________


In [9]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 8000/32,
                         epochs = 25, 
                         validation_data = test_set,
                         validation_steps = 2000/32,
                         verbose = 1)

Epoch 1/25
250/250 [==============================] - 107s 428ms/step - loss: 0.6853 - acc: 0.5824 - val_loss: 0.6372 - val_acc: 0.6585
Epoch 2/25
250/250 [==============================] - 105s 419ms/step - loss: 0.6374 - acc: 0.6395 - val_loss: 0.6059 - val_acc: 0.6900
Epoch 3/25
250/250 [==============================] - 104s 415ms/step - loss: 0.6025 - acc: 0.6804 - val_loss: 0.5736 - val_acc: 0.7120
Epoch 4/25
250/250 [==============================] - 105s 418ms/step - loss: 0.5562 - acc: 0.7181 - val_loss: 0.5478 - val_acc: 0.7140
Epoch 5/25
250/250 [==============================] - 105s 419ms/step - loss: 0.5320 - acc: 0.7375 - val_loss: 0.5331 - val_acc: 0.7425
Epoch 6/25
250/250 [==============================] - 105s 418ms/step - loss: 0.4932 - acc: 0.7609 - val_loss: 0.4884 - val_acc: 0.7725
Epoch 7/25
250/250 [==============================] - 104s 416ms/step - loss: 0.4706 - acc: 0.7746 - val_loss: 0.5072 - val_acc: 0.7565
Epoch 8/25
250/250 [============================

By introducing a new Convolution layer, I was able to have the test accuracy as 79.40%